# Fluxos de rede

Muitos problemas computacionais podem ser representados por um grafo que consiste em nós e arcos entre eles. Um exemplo muito importante são problemas de fluxo de rede, que envolvem o transporte de mercadorias ou materiais através de uma rede, como um sistema ferroviário. 

Você pode representar um fluxo de rede por um gráfico cujos nós são cidades e cujos arcos são linhas ferroviárias entre eles. (Eles são chamados de fluxos porque suas propriedades são semelhantes às da água fluindo através de uma rede de tubos.)

Uma restrição importante nos fluxos de rede é que cada arco tem uma **capacidade** - a quantidade máxima que pode ser transportada através do arco em um período de tempo fixo. 

## Fluxos Máximos

O problema de fluxo máximo é determinar a quantidade total máxima que pode ser transportada por todos os arcos da rede, sujeita às restrições de capacidade.

A seguir, veremos um exemplo de problema de fluxo máximo. O problema é definido pelo seguinte grafo, que representa uma rede de transporte:

<img src="./img/fluxo-maximo.png" width="50%">

Queremos transportar material do nó 0 (a fonte) para o nó 4 (o sumidouro). 

Os números próximos aos arcos são suas capacidades - a capacidade de um arco é a quantidade máxima que pode ser transportada por ele em um período de tempo fixo. As capacidades são as restrições para o problema.

Um fluxo é uma atribuição de um número não negativo a cada arco (a quantidade de fluxo) que satisfaz a seguinte regra de conservação de fluxo:

    Em cada nó, que não seja a fonte ou o sumidouro, o fluxo total de todos os arcos que levam ao nó é igual ao fluxo total de todos os arcos que saem dele.
    
O problema do fluxo máximo é encontrar um fluxo para o qual a soma dos valores de fluxo para toda a rede seja a maior possível.

A seguir veremos um programa em C++ para encontrar o fluxo máximo da fonte (nó 0) para o sumidouro (nó 4).

In [ ]:
#include <iostream>
#include <iomanip>
#include "setup.h"
#include "ortools/graph/max_flow.h"

using namespace operations_research;

### Defina os dados

Você pode representar o grafo para o problema com três vetores, para os nós iniciais, nós finais e capacidades dos arcos. O comprimento de cada vetor é igual ao número de arcos no grafo. Para cada $i$, arco $i$ vai de start_nodes[i] a end_nodes[i], e sua capacidade é dada por capacities[i]. 

A próxima seção mostra como criar os arcos usando esses dados.

In [ ]:
int numNodes = 5;
int numArcs = 9;
int start_nodes[] = {0, 0, 0, 1, 1, 2, 2, 3, 3};
int end_nodes[] = {1, 2, 3, 2, 4, 3, 4, 2, 4};
int capacities[] = {20, 30, 10, 40, 30, 10, 20, 5, 20};

### Declare o solucionador e adicione os arcos

Para resolver o problema, usamos o solucionador [SimpleMaxFlow](https://developers.google.com/optimization/reference/graph/max_flow/SimpleMaxFlow).

Para cada o nó inicial e nó final, criamos um arco do nó inicial ao nó final com a capacidade fornecida, usando o método AddArcWithCapacity. As capacidades são as restrições para o problema.

In [ ]:
SimpleMaxFlow maxFlow;

for (int i = 0; i < numArcs; ++i)
{
  int arc = maxFlow.AddArcWithCapacity(start_nodes[i], end_nodes[i], capacities[i]);
}
int source = 0;
int sink = numNodes - 1;

### Chame o solucionador e exiba os resultados

Agora que todos os arcos foram definidos, tudo o que resta é invocar o solucionador e exibir os resultados. Invocamos o método Solve(), fornecendo a fonte (0) e o sumidouro (4) e, a seguir, exibimos o fluxo em cada arco.

In [ ]:
std::cout << "Solving max flow with " << numNodes << " nodes, and " 
    << numArcs << " arcs, source=" << source << ", sink=" << sink << std::endl;

// Find the maximum flow between node 0 and node 4.
int solveStatus = maxFlow.Solve(source, sink);
if (solveStatus == SimpleMaxFlow::OPTIMAL) {
  std::cout << "Max. flow: " << maxFlow.OptimalFlow() << std::endl;
  std::cout << "  Arc     Flow / Capacity" << std::endl;
  for (int i = 0; i < numArcs; ++i) {
    std::cout << maxFlow.Tail(i) << " -> " <<
                      maxFlow.Head(i) << "    " <<
                      maxFlow.Flow(i) << "  /  " <<
                      maxFlow.Capacity(i) << std::endl;
  }
}
else {
  std::cout << "Solving the max flow problem failed. Solver status: " << solveStatus;
}

## Fluxos de Custo Mínimo

O problema de fluxo de custo mínimo está intimamente relacionado ao problema de fluxo máximo com a diferença que cada arco no gráfico tem um custo unitário para transportar material através dele. O problema é encontrar um fluxo com o menor custo total.

O problema de fluxo de custo mínimo também possui nós especiais, chamados de nós de suprimento ou nós de demanda, que são semelhantes à origem e ao sumidouro no problema de fluxo máximo. O material é transportado de nós de suprimento para nós de demanda.

- Em um nó de suprimento, uma quantidade positiva - o suprimento - é adicionado ao fluxo. Um suprimento pode representar a produção naquele nó, por exemplo.

- Em um nó de demanda, uma quantidade negativa - a demanda - é retirada do fluxo. Uma demanda pode representar o consumo naquele nó, por exemplo.

Por conveniência, assumiremos que todos os nós, exceto os nós de suprimento ou demanda, têm suprimento (e demanda) zero.

Para o problema de fluxo de custo mínimo, temos a seguinte regra de conservação de fluxo, que leva em consideração os suprimentos e as demandas:

    Em cada nó, o fluxo total que sai do nó menos o fluxo total que chega ao nó é igual ao suprimento (ou demanda) nesse nó.
    
O grafo abaixo mostra um problema de fluxo de custo mínimo. Os arcos são rotulados com pares de números: 
- o primeiro número é a capacidade e o segundo número é o custo. 
- os números entre parênteses próximos aos nós representam suprimentos ou demandas. 

O nó 0 é um nó de suprimento com fornecimento 20, enquanto os nós 3 e 4 são nós de demanda, com demandas -5 e -15, respectivamente.

<img src="./img/fluxo-custo-min.png" width="50%">


In [1]:
#include <iostream>
#include <iomanip>
#include "setup.h"
#include "ortools/graph/min_cost_flow.h"

using namespace operations_research;

## Defina os dados

O código a seguir define os dados para o problema. Nesse caso, há quatro matrizes para os nós iniciais, nós finais, capacidades e custos unitários. Novamente, o comprimento das matrizes é o número de arcos no grafo.

In [2]:
int numNodes = 5;
int numArcs = 9;

int startNodes[] = {0, 0, 1, 1, 1, 2, 2, 3, 4};
int endNodes[] = {1, 2, 2, 3, 4, 3, 4, 4, 2};
int capacities[] = {15, 8, 20, 4, 10, 15, 4, 20, 5};
int unitCosts[] = {4, 4, 2, 2, 6, 1, 3, 2, 3};

// Define an array of supplies at each node.
int supplies[] = {20, 0, 0, -5, -15};

## Declare o solucionador e adicione os arcos

Para resolver o problema, usamos o solucionador [SimpleMinCostFlow](https://developers.google.com/optimization/reference/graph/min_cost_flow/SimpleMinCostFlow).

Para cada nó inicial e nó final, criamos um arco do nó inicial ao nó final com a capacidade e custo unitário dados, usando o método **AddArcWithCapacityAndUnitCost**.

O método **SetNodeSupply** do solucionador cria um vetor de suprimentos para os nós.

In [3]:
SimpleMinCostFlow minCostFlow;

// Add each arc.
for (int i = 0; i < numArcs; ++i) {
  int arc = minCostFlow.AddArcWithCapacityAndUnitCost(startNodes[i], endNodes[i],
                                       capacities[i], unitCosts[i]);
}

 // Add node supplies.
for (int i = 0; i < numNodes; ++i) {
  minCostFlow.SetNodeSupply(i, supplies[i]);
}

## Chame o solucionador e exiba os resultados

Agora que todos os arcos foram definidos, tudo o que resta é invocar o solucionador e exibir os resultados. Chamamos o método Solve () e, em seguida, exibimos o fluxo e o custo em cada arco.

In [4]:
int solveStatus = minCostFlow.Solve();
if (solveStatus == MinCostFlow::OPTIMAL) {
  auto optimalCost = minCostFlow.OptimalCost();
  std::cout << "Minimum cost: " << optimalCost << std::endl;

  std::cout << " Edge   Flow / Capacity  Cost" << std::endl;
  for (int i = 0; i < numArcs; ++i) {
    auto cost = minCostFlow.Flow(i) * minCostFlow.UnitCost(i);
    std::cout << minCostFlow.Tail(i) << " -> " <<
                      minCostFlow.Head(i) << "  " <<
                      minCostFlow.Flow(i) << "  / " <<
                      minCostFlow.Capacity(i) << "       " <<
                      cost << std::endl;
  }
}
else {
  std::cout << "Solving the min cost flow problem failed. Solver status: " << solveStatus;
}

Minimum cost: 150
 Edge   Flow / Capacity  Cost
0 -> 1  12  / 15       48
0 -> 2  8  / 8       32
1 -> 2  8  / 20       16
1 -> 3  4  / 4       8
1 -> 4  0  / 10       0
2 -> 3  12  / 15       12
2 -> 4  4  / 4       12
3 -> 4  11  / 20       22
4 -> 2  0  / 5       0
